In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from darts.models import NBEATSModel

In [4]:
import dask.dataframe as dd

In [5]:
from darts.timeseries import TimeSeries

In [6]:
import gc

In [7]:
import pickle

In [8]:
# import sys
# sys.path.append("../data/")

In [9]:
# import ubiquant

##### Global Variables ####

In [10]:
gc.collect()

22

In [11]:
DATA_DIR = '../data/parquet/'

In [12]:
input_chunk_length = 30
output_chunk_length = 1

In [13]:
features = [f'f_{i}' for i in range(300)]

#### Read Data #####

In [14]:
# from IPython.display import clear_output
# clear_output()

In [15]:
# %%time 
# train = dd.read_csv(DATA_DIR+'train.csv')

In [16]:
# %%time
# train_df = train.compute()

In [17]:
%%time
train_df = pd.read_parquet(DATA_DIR+'train_low_mem.parquet')

CPU times: user 27.7 s, sys: 9.77 s, total: 37.5 s
Wall time: 6.4 s


In [18]:
train_df.head()

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624


In [19]:
####### Filter out only Fold 1 data ###########

In [20]:
with open('../input/folds.pickle', 'rb') as f:
    folds = pickle.load(f)

In [21]:
train_indxs = folds[0]['train']
test_indxs = folds[0]['test']

In [22]:
test_df = train_df[train_df.index.isin(test_indxs)].reset_index(drop=True)
train_df = train_df[train_df.index.isin(train_indxs)].reset_index(drop=True)

In [23]:
gc.collect()

0

In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531075 entries, 0 to 531074
Columns: 304 entries, row_id to f_299
dtypes: float32(301), object(1), uint16(2)
memory usage: 615.9+ MB


In [25]:
# train_df = train_df.sample(frac=0.2)

In [26]:
investment_ids = train_df.investment_id.unique().tolist()

In [27]:
count_df = train_df.groupby('investment_id',as_index=False).agg(c = ('time_id', len))

In [28]:
count_df.c.describe()

count    3577.000000
mean      148.469388
std        52.384007
min         2.000000
25%       116.000000
50%       170.000000
75%       191.000000
max       206.000000
Name: c, dtype: float64

In [29]:
to_exclude_ids = count_df[count_df.c<input_chunk_length+2].investment_id.unique().tolist()

In [30]:
train_df.shape

(531075, 304)

In [31]:
#### past covariates will be features #####

##### Target and past covariates for each timeseries ######

In [32]:
data_dict = {}

In [33]:
def create_data_dict():
    for investment_id in investment_ids:
        if investment_id not in to_exclude_ids:
            df = train_df[train_df.investment_id == investment_id]

            data_dict[str(investment_id)] = {}
            data_dict[str(investment_id)]['targets'] = TimeSeries.from_dataframe(df,time_col = 'time_id', value_cols = 'target')
            data_dict[str(investment_id)]['past_covariates'] = TimeSeries.from_dataframe(df,time_col = 'time_id', value_cols = features)
        
    return

In [34]:
%%time
create_data_dict()

CPU times: user 9.43 s, sys: 37.9 ms, total: 9.47 s
Wall time: 9.47 s


### Model ####

In [35]:
model = NBEATSModel(input_chunk_length=input_chunk_length, 
                             output_chunk_length=output_chunk_length)

In [36]:
train_series = [data_dict[k]['targets'] for k in data_dict.keys()]

In [37]:
train_past_covariates_series = [data_dict[k]['past_covariates'] for k in data_dict.keys()]

In [38]:
gc.collect()

0

In [39]:
model.fit(train_series[:10],past_covariates=train_past_covariates_series[:10], 
                 epochs=5, 
                 verbose=True)

[2022-01-31 00:35:47,963] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 1710 samples.
[2022-01-31 00:35:47,963] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 1710 samples.
[2022-01-31 00:35:48,326] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 32-bits; casting model to float32.
[2022-01-31 00:35:48,326] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 32-bits; casting model to float32.


  0%|          | 0/5 [00:00<?, ?it/s]

### Inference #####

In [40]:
test_df

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,13_168,13,168,-0.069346,0.471798,-1.888772,1.528517,-0.498815,-0.565286,0.787917,...,1.401624,1.529582,0.048327,-0.831065,0.854191,-0.617494,-1.367622,-0.002489,1.055625,-0.710585
1,14_168,14,168,-0.229911,0.214289,-1.990782,1.544363,-0.511079,-0.569731,0.443899,...,0.302519,-0.848933,0.341934,-1.440869,-0.938487,-0.441435,1.106537,-0.535386,0.779353,-0.729012
2,15_168,15,168,-0.198795,-0.363923,-2.102880,1.597715,-0.568584,-0.565837,0.473172,...,-0.472090,0.446282,-0.653184,-1.878950,0.409130,-0.419934,0.698586,0.295534,0.775944,-0.746361
3,16_168,16,168,0.006404,0.014698,-1.890795,1.606887,-0.616617,-0.561555,0.557014,...,0.086683,0.555996,-0.730524,0.678793,-0.881622,-0.578475,-0.249035,0.031241,1.019552,-0.730968
4,17_168,17,168,0.163887,0.220540,-1.846194,1.694260,-0.651389,-0.557331,0.627228,...,-1.062184,0.514449,-0.662535,0.523865,0.740017,-0.561478,1.117373,0.063991,0.904364,-0.735987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522061,1211_890,1211,890,-0.208120,0.188757,0.042285,0.000000,1.583300,-0.176820,-0.761516,...,-0.144227,-0.776686,-1.242701,0.165202,-0.676523,-0.626533,0.714443,-0.373408,-1.984618,-0.414150
522062,1215_85,1215,85,-3.547906,-1.853507,1.620525,0.000000,10.258781,0.244644,-0.110384,...,-2.165493,-0.694756,1.247323,0.769126,0.966756,2.331125,-1.303493,1.633453,2.820497,2.505889
522063,1215_905,1215,905,-1.416078,0.559272,0.318800,0.000000,3.273175,0.761414,1.991194,...,-0.979614,-0.694756,-0.071421,0.769126,0.966756,2.679727,1.138049,-0.651654,0.000000,5.052356
522064,1215_2558,1215,2558,-1.296170,0.747612,0.029528,0.000000,-0.278775,1.447511,-1.339483,...,1.316046,-0.694756,-0.514473,-1.300175,0.966756,1.476327,1.138049,-0.391025,0.000000,2.544184
